**Copyright 2024 Google LLC.**

Licensed under the Apache License, Version 2.0 (the "License");

# Prepare

In [ ]:
%pip install -U -q google-ads-api-report-fetcher
%pip install -U -q rich

In [ ]:
from google.colab import data_table
from gaarf.api_clients import GoogleAdsApiClient
from gaarf.query_executor import AdsReportFetcher
import rich
from gaarf.io.writer import StdoutWriter
import os
import logging
from google.colab import userdata
data_table.enable_dataframe_formatter()
logger = logging.getLogger()
logging.getLogger().setLevel(level=logging.INFO) # set to DEBUG to see more detailed output
logging.getLogger('google.ads.googleads.client').setLevel(level=logging.WARNING)
#from google.colab import auth
#auth.authenticate_user()

ads_config = {
  "developer_token": "",
  "client_id": "",
  "client_secret": "",
  "refresh_token": "",
  "login_customer_id": "",
  "customer_id": "",
  "use_proto_plus": True
}
ads_config_yaml = """
"""
#TODO: use Colab userdata:
#   from google.colab import userdata
#   API_KEY=userdata.get('API_KEY')
if len(ads_config_yaml.strip()):
  client = GoogleAdsApiClient(yaml_str=ads_config_yaml)
elif os.path.isfile('./google-ads.yaml'):
  client = GoogleAdsApiClient(path_to_config='google-ads.yaml')
elif ads_config.get('developer_token') and ads_config.get('refresh_token') :
  client = GoogleAdsApiClient(config_dict=ads_config)
else:
  raise Exception('Please provide Google Ads credentials, either upload google-ads.yaml or put them into the ads_config_yaml variable as YAML text or in ads_config as JSON')
report_fetcher = AdsReportFetcher(client)

# Choose Ads account

Please enter a MCC ID or a leave CID in the field below. We'll use it to fetch all campaigns with adgroups and keywords from which I can choose one to use for migrating to search themes in PMAX.

In [ ]:
# @title Expand MCC { run: "auto" }
MCC_ID = "6368728866" # @param {type:"string"}
customer_ids = report_fetcher.expand_mcc(str(MCC_ID))
rich.print(customer_ids)
CIDs = ",".join([str(cid) for cid in customer_ids])
print('You can replace the MCC in the form above onto one of the expanded CIDs or leave it as is')

# Choose target PMAX campaign

In [ ]:
print(f"Fetching PMAX campaign for the following accounts: {customer_ids}")
query = """
SELECT
  customer.id,
  customer.descriptive_name as customer_name,
  campaign.id,
  campaign.name,
  asset_group.id AS asset_group_id,
  asset_group.name AS asset_group_name,
  metrics.cost_micros as metrics_cost,
  campaign.advertising_channel_type,
  campaign.status
FROM asset_group
WHERE campaign.advertising_channel_type = 'PERFORMANCE_MAX'
AND asset_group.status = 'ENABLED'
AND campaign.status = 'ENABLED'
"""
report = report_fetcher.fetch(query, customer_ids)
writer = StdoutWriter(page_size=100)
#writer.write(report[['customer_id', 'customer_name', 'campaign_id', 'campaign_name', 'asset_group_id', 'asset_group_name', 'metrics_cost']], 'PMAX AssetGroups')
report.to_pandas()

In [ ]:
# @title Chosing a PMAX target AssetGroup
ASSET_GROUP_ID = "6466108739" # @param {type:"string"}
pmax_customer_id = ""
pmax_campaign_id = ""

if not ASSET_GROUP_ID:
  raise Exception('Please enter an asset group id.')

# find a CID of the selected asset_group
for row in report:
  if str(row['asset_group_id']) == ASSET_GROUP_ID:
    pmax_customer_id = str(row['customer_id'])
    pmax_campaign_id = str(row['campaign_id'])
print(f'target for migration: CID={pmax_customer_id}, Campaign={pmax_campaign_id}, AssetGroup={ASSET_GROUP_ID}')

# Choose source search campaign/ad group

In [ ]:
def getKeywords(customer_id):
  query = """SELECT
    customer.id,
    customer.descriptive_name,
    campaign.id,
    campaign.name,
    ad_group.id,
    ad_group.name,
    ad_group_criterion.keyword.text as keyword,
    metrics.clicks,
    metrics.conversions,
    metrics.cost_micros
  FROM keyword_view
  WHERE ad_group.type = SEARCH_STANDARD
    AND campaign.status = ENABLED
    AND ad_group_criterion.status = ENABLED
    AND metrics.clicks > 0
    """
  report = report_fetcher.fetch(query, customer_id)
  return report


customer_ids = report_fetcher.expand_mcc(str(MCC_ID))
customers = {}
for customer_id in customer_ids:
  report = getKeywords(customer_id)
  if report:
    customers[customer_id] = report
print('Customers with adgroups having keywords with non-zero clicks:')
rich.print(list(customers.keys()))
print()

for customer_id in customers.keys():
  report = customers[customer_id]
  print('Customer:', customer_id)
  print(report.to_list())
  campaigns = {}
  adgroups = {}
  keywords = {}
  for row in report:
    campaigns[row['campaign_id']] = row['campaign_name']
    adgroups[row['ad_group_id']] = row['ad_group_name']
    keywords[row['keyword']] = row['metrics_clicks']
  print('Campaigns:')
  rich.print(campaigns)
  print('AdGroups:')
  rich.print(adgroups)
  print()


Please enter either a campaign id or a adgroup id.

In [ ]:
# @title Parameters
CAMPAIGN_ID = "" # @param {type:"string"}
AD_GROUP_ID = "121105014899; 74997665417 " # @param {type:"string"}



In [ ]:
if not CAMPAIGN_ID and not AD_GROUP_ID:
  raise Exception('Please enter either a campaign id or a adgroup id.')
CAMPAIGN_ID_LIST = [id.strip() for id in CAMPAIGN_ID.split(';')]
AD_GROUP_ID_LIST = [id.strip() for id in AD_GROUP_ID.split(';')]
keywords = []
for customer_id in customers.keys():
  report = customers[customer_id]
  for row in report:
    if CAMPAIGN_ID:
      if str(row['campaign_id']) in CAMPAIGN_ID_LIST:
        #rich.print(row['keyword'])
        keywords.append(row['keyword'])
    elif AD_GROUP_ID:
      if str(row['ad_group_id']) in AD_GROUP_ID_LIST:
        #rich.print(row['keyword'])
        keywords.append(row['keyword'])
print(f'Keywords for {"Campaign " + CAMPAIGN_ID if CAMPAIGN_ID else "AdGroup " + AD_GROUP_ID} ')
rich.print(keywords)


# Migration


## Case 1: Move all keywords as themes distributed equally

When to use it: initial structure of adgroups in search campaign(s) makes not much sense or you don't keep it. In this case we'll grad all keywords from all adroups in the context and split them equally by buckets of 25 and create a new PMAX AssetGroup for each of them.

In [ ]:
from uuid import uuid4
from google.ads.googleads.errors import GoogleAdsException

logging.getLogger('google.ads.googleads.client').setLevel(level=logging.WARN)

def fetch_asset_group_audience(customer_id, asset_group_id):
  query = f"""SELECT
    asset_group_signal.audience.audience as audience
  FROM asset_group_signal
  WHERE asset_group.id = {asset_group_id}
  """
  report = report_fetcher.fetch(query, customer_id)
  values = report.to_list()
  audiences = [val for val in values if val]
  print('audience:', audiences)
  return audiences[0] if audiences else None


def clone_asset_group(customer_id, campaign_id, asset_group_id):
  """clone AssetGroup with id = ASSET_GROUP_ID"""
  query = f"""SELECT
    asset_group.id,
    asset_group.final_mobile_urls,
    asset_group.final_urls,
    asset_group.name,
    asset_group.path1,
    asset_group.path2,
    asset_group_asset.field_type as field_type,
    asset.id,
    asset.resource_name,
  FROM asset_group_asset
  WHERE asset_group.id = {asset_group_id}
    AND asset_group_asset.status = ENABLED
    """
  # NOTE: aleternately we can use `asset_group_asset.primary_status = ELIGIBLE`
  report = report_fetcher.fetch(query, customer_id)
  #rich.print(report.to_list())

  campaign_service = client.client.get_service("CampaignService")
  asset_service = client.client.get_service("AssetService")
  asset_group_service = client.client.get_service("AssetGroupService")
  asset_group_resource_name = asset_group_service.asset_group_path(
      customer_id,
      "-1",
  )
  operations = []

  # Create the AssetGroup
  mutate_operation = client.client.get_type("MutateOperation")
  asset_group = mutate_operation.asset_group_operation.create
  asset_group.name = f"PMax-Themer auto-created asset group #{uuid4()}"
  asset_group.campaign = campaign_service.campaign_path(
      customer_id, campaign_id
  )

  urls = report.to_dict(key_column='asset_group_id', value_column='asset_group_final_urls', value_column_output='list')[int(asset_group_id)]
  urls = [item for sublist in urls for item in sublist][1:]
  asset_group.final_urls.append(urls[0])
  urls = report.to_dict(key_column='asset_group_id', value_column='asset_group_final_mobile_urls', value_column_output='list')[int(asset_group_id)]
  urls = [item for sublist in urls for item in sublist][1:]
  if urls:
    asset_group.final_mobile_urls.append(urls[0])
  asset_group.status = client.client.enums.AssetGroupStatusEnum.PAUSED
  asset_group.resource_name = asset_group_resource_name
  operations.append(mutate_operation)

  # if the source AssetGroup has an Audience, reuse it in the new AssetGroup
  audience_resource = fetch_asset_group_audience(customer_id, asset_group_id)
  if audience_resource:
    mutate_operation = client.client.get_type("MutateOperation")
    operation = mutate_operation.asset_group_signal_operation.create
    operation.asset_group = asset_group_resource_name
    operation.audience.audience = audience_resource
    operations.append(mutate_operation)

  # copy all assets
  for row in report:
    # Create an AssetGroupAsset to link the Asset to the AssetGroup.
    mutate_operation = client.client.get_type("MutateOperation")
    asset_group_asset = mutate_operation.asset_group_asset_operation.create
    asset_group_asset.field_type = row['field_type'] #client.client.enums.AssetFieldTypeEnum.
    asset_group_asset.asset_group = asset_group.resource_name
    asset_group_asset.asset = row['asset_resource_name']
    operations.append(mutate_operation)

  logging.debug(operations)

  # creating a new asset_group with asset_group_asset per each asset in the source asset_group
  googleads_service = client.client.get_service("GoogleAdsService")
  response = googleads_service.mutate(
      customer_id=customer_id, mutate_operations=operations,
  )
  logging.debug(response)
  # Extract the new AssetGroup's resource_name and return
  # mutate_operation_responses {
  #   asset_group_result {
  #     resource_name: "customers/xxxx/assetGroups/yyy"
  #   }
  # }
  #
  #if response.partial_failure_error:

  for result in response.mutate_operation_responses:
      if result._pb.HasField("asset_group_result"):
        logging.info(f'Created AssetGroup in campaing {campaign_id} with name="{asset_group.name}", resource_name={result.asset_group_result.resource_name}')
        return result.asset_group_result.resource_name

  raise Exception('Could not find asset_group_result in mutate response')


def create_assetgroups_with_signals(keywords):
  chunk_size = 25
  idx = 0
  asset_group_resource_name = ''
  created_ag = []
  while idx < len(keywords):
    # Extract a chunk of keywords
    search_theme_chunk = keywords[idx: idx + chunk_size]
    # create a clone of AssetGroup as a container for new search themes (we don't except a failure here)
    if not asset_group_resource_name:
      asset_group_resource_name = clone_asset_group(pmax_customer_id, pmax_campaign_id, ASSET_GROUP_ID)
    else:
      print('Reusing the AssetGroup {asset_group_resource_name}')
    mutate_operations = []
    print(f'Creating search themes for the AssetGroup {asset_group_resource_name} with {len(search_theme_chunk)} keywords:\n', search_theme_chunk)
    for search_theme in search_theme_chunk:
      mutate_operation = client.client.get_type("MutateOperation")
      operation = mutate_operation.asset_group_signal_operation.create
      operation.asset_group = asset_group_resource_name
      operation.search_theme.text = search_theme
      mutate_operations.append(mutate_operation)

    #logging.debug(mutate_operations)
    request = client.client.get_type("MutateGoogleAdsRequest")
    request.customer_id = str(pmax_customer_id)
    request.mutate_operations = mutate_operations
    #request.partial_failure = True
    banned_themese = []
    try:
      response = client.ads_service.mutate(request)
      # Move to the next chunk if the API call is successful
      idx += chunk_size
      created_ag.append(asset_group_resource_name)
      asset_group_resource_name = ''
      logging.debug(response)
      print('AssetGroup successfully updated with search themes')
    except GoogleAdsException as ex:
      # If API call is not successful, remove the banned keywords received in the response\
      # NOTE: we'll reuse the created AssetGroup (asset_group_resource_name)
      for error in ex.failure.errors:
        print(f'\tError with message "{error.message}".')
        #if error.location:
        #  for field_path_element in error.location.field_path_elements:
        #    print(f"\t\tOn field: {field_path_element.field_name}")
        if error.error_code.asset_group_signal_error == 3:  # SEARCH_THEME_POLICY_VIOLATION
          banned_themese.append(error.trigger.string_value)
          try:
            print(f'Search theme "{error.trigger.string_value}" was banned by policy "{error.details.policy_violation_details.key.policy_name}" (violating: "{error.details.policy_violation_details.key.violating_text}")')
          except BaseException as e:
            print('WARNING: failed to print policy violation: ', e)
      if len(banned_themese) > 0:
        print("The following keywords have been banned by policy check and were not uploaded as search themes:\n", banned_themese)
        # removing banned keywords from the list
        keywords = list(filter(lambda k: k not in banned_themese, keywords))
      else:
        print(f'WARNING: Unknow error, mutate operation with AsseetGroup and signals (themese) failed but not because of banned themse (or we failed to parse the result). Original error is following')
        print(ex)
  return created_ag

# deduplicate keywords
keywords = list(set(keywords))
print(f'Creating PMax search themes for {len(keywords)} keywords')
created_ag = create_assetgroups_with_signals(keywords)
print(f'Done. Created {len(created_ag)} Asset Groups')
rich.print(created_ag)


## Case 2: Move keywords with structure of adgroups

When to use it: initial structure of adgroups in search campaign(s) is important and we want to keep it.

In [ ]:
from uuid import uuid4
from google.ads.googleads.errors import GoogleAdsException